In [ ]:
import pandas as pd

import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns


from utils import read_seller_delivery

In [ ]:
check1 = df[df['seller_id'] == '7c67e1448b00f6e969d365cea6b010ab']
# Create a box plot grouping data by day of the week
fig, ax = plt.subplots(figsize=(10, 6)) #

sns.boxplot(y='total_days', data=check1, ax = ax)
plt.title('Seller Shipping Time')
plt.show()